In [ ]:
import pandas as pd
import numpy as np
import bnlearn as bn
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel('exoplanets.xls')
df.head()

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(20, 5))
bins = 100
df['radius (m)'].plot.hist(bins=bins, ax=axs[0])
df['mass (kg)'].plot.hist(bins=bins, ax=axs[1])
df['gravity'].plot.hist(bins=bins, ax=axs[2])
df['avg_temp (C)'].plot.hist(bins=bins, ax=axs[3])
df['rotational_period (h)'].plot.hist(bins=bins, ax=axs[4])

In [ ]:
variables = [
    ('radius (m)', 'radius'),
    ('mass (kg)', 'mass'),
    ('gravity', 'gravity'),
    ('avg_temp (C)', 'avg_temp'),
    ('rotational_period (h)', 'rotational_period'),
]

for old_name, new_name in variables:
    q1, q2, qmax = df[old_name].quantile([1/3, 2/3, 1])
    series = pd.cut(df[old_name], [0, q1, q2, qmax], labels=['small', 'medium', 'large'])
    series.name = new_name
    del df[old_name]
    df[new_name] = series

df.head()

In [ ]:
exo_dag = bn.structure_learning.fit(df, methodtype='hc', scoretype='bic')

In [ ]:
print(exo_dag['adjmat'])
bn.plot(exo_dag, interactive=True)

In [ ]:
adjmat = exo_dag['adjmat']

edges = [
  ('radius', 'mass'),
  ('mass', 'gravity'),
  ('atmosphere', 'avg_temp'),
  ('water', 'avg_temp'),
  ('gravity', 'atmosphere'),
  ('water', 'life'),
  ('oxygene', 'life'),
  ('atmosphere', 'oxygene'),
  ('avg_temp', 'life'),
]

false_edges = [
  ('atmosphere', 'mass'),
]

for (source, target) in edges:
  adjmat.loc[target, source] = False
  adjmat.loc[source, target] = True

for (source, target) in false_edges:
  adjmat.loc[target, source] = False
  adjmat.loc[source, target] = False

adjmat.fillna(False, inplace=True)
adjmat

In [ ]:
exo_dag['adjmat'] = adjmat
bn.plot(exo_dag, interactive=True)

In [ ]:
dag_names = set(adjmat.columns)

df_names = set(df.columns)

print(dag_names)
print(df_names)
print(dag_names - df_names)

In [ ]:
exo_model = bn.parameter_learning.fit(exo_dag, df, methodtype='maximumlikelihood')

In [ ]:
bn.inference.fit(exo_model, variables=['mass', 'avg_temp'], evidence={'life': 'YES'})

In [ ]:
dir(exo_model['model'])